In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.metrics import classification_report

# CLS

# `compute_distance_sums` Function

The `compute_distance_sums` function calculates the cumulative distance between a given test image embedding and a set of stored memory embeddings. The computed distances are aggregated based on class labels, providing a per-class similarity score.

## Parameters

- **selected_test_image_embedding** (`numpy array`): The embedding vector of the test image.
- **memory_embeddings** (`list of numpy arrays`): A collection of embeddings stored in memory.
- **memory_labels** (`list of int`): Class labels corresponding to `memory_embeddings`.
- **predicted_softmax_label** (`numpy array`, optional): A probability distribution over classes, from the finetuned model.
- **static_distance** (`float`, default=`1`): A scaling factor applied to `predicted_softmax_label`.
- **distance_metric** (`str`, default=`'cosine'`): Specifies the distance metric to use. Options:
  - `'cosine'`: Uses cosine similarity.
  - `'euclidean'`: Uses a modified Euclidean distance (`1 / (1 + d)`).
- **num_classes** (`int`, default=`10`): Total number of possible classes.

## Returns

- **distance_sums** (`numpy array`): An array of length `num_classes`, where each index represents the cumulative similarity score for that class.

## Methodology

### 1. One-Hot Encoding of Labels
- A one-hot encoded matrix of shape `(len(memory_labels), num_classes)` is created to represent the class labels.
- If any label is out of range, an error is raised.

### 2. Distance Computation
- Each memory embedding is compared to the `selected_test_image_embedding` using the specified `distance_metric`.
  - **Cosine similarity** is used if `distance_metric='cosine'`.
  - A **normalized Euclidean distance** (`1 / (1 + d)`) is used if `distance_metric='euclidean'`.
- The computed distance is multiplied by the corresponding one-hot encoded label and added to `distance_sums`.

### 3. Adjustment Using Softmax Prediction
- If `predicted_softmax_label` is provided, it is scaled by `static_distance` and added to `distance_sums`.

In [2]:
def compute_distance_sums(selected_test_image_embedding, memory_embeddings, memory_labels, predicted_softmax_label= None, static_distance = 1, distance_metric='cosine', num_classes=10):

    distance_sums = np.zeros(num_classes)

    # Initialize the one-hot vectors based on memory_labels
    memory_labels_onehot = np.zeros((len(memory_labels), num_classes))

    # Create the one-hot encoding for each memory label
    for i, label in enumerate(memory_labels):
        if label < num_classes:
            memory_labels_onehot[i, label] = 1
        else:
            raise ValueError(f"Label {label} is out of range. Expected between 0 and {num_classes - 1}.")

    # Loop over each memory embedding and compute distances
    for i, memory_embedding in enumerate(memory_embeddings):
        # One-hot encode the memory label
        memory_label_onehot = memory_labels_onehot[i]
        
        # When two embeddings are identical the cosine = 1 and the Euclidean is 0
        # when two embeddings are very different then cosine = -1 and euclidean = inf
        # Compute the 1/(1+d) for Euclidean in order to be 1 for identical and 0 for very different,
        # Compute the distance (Cosine or Euclidean based on the parameter)
        if distance_metric == 'cosine':
            dist = cosine_similarity([selected_test_image_embedding], [memory_embedding])[0][0]
        elif distance_metric == 'euclidean':
            dist = 1/(1+euclidean_distances([selected_test_image_embedding], [memory_embedding])[0][0]) # 1/(1+d)
        else:
            raise ValueError("Unsupported distance metric. Choose 'cosine' or 'euclidean'.")
        
        # print(f'######Iteration {i}#########')
        # print(memory_labels[i])
        # print(memory_label_onehot)
        # print(dist)
        # print(f'Before: {distance_sums}')
        # Add the distance to the initialized array
        distance_sums += memory_label_onehot * dist
        # print(f'After: {distance_sums}')
        # print('###############')

    
    # Final calculation based on the distance metric
    if predicted_softmax_label is not None:
        # print('########## Final #########')
        
        # Add the predicted value weighted by static distance
        distance_sums += predicted_softmax_label * static_distance
        
        # print(predicted_softmax_label * static_distance)
        # print(f'Final: {distance_sums}')

    return distance_sums

# `get_predicted_class` Function

The `get_predicted_class` function determines the predicted class based on the cumulative similarity scores (`distance_sums`) computed by the `compute_distance_sums` function.

In [3]:
def get_predicted_class(distance_sums):
    index = np.argmax(distance_sums)
    return index

# `load_data_cls` Function

The `load_data_cls` function loads data from a `.npz` file, which contains test and neighbor class embeddings along with their corresponding labels.

In [15]:
def load_data_cls(file_path: str):
    # Load the .npz file
    data = np.load(file_path)

    # for key in data.keys():
    #     print(f"{key}: {data[key].shape}")
    
    # Extract the arrays
    test_cls = data['test_cls']
    test_labels = data['test_labels']
    neighbor_cls = data['neighbor_cls']
    neighbor_labels = data['neighbor_labels']
    print("Data loaded successfully.")
    
    return test_cls, test_labels, neighbor_cls, neighbor_labels

# `load_softmax_predictions` Function

The `load_softmax_predictions` function loads softmax predictions from a `.npz` file. These predictions represent the probability distribution over classes for a set of inputs.

In [10]:
def load_softmax_predictions(file_path: str):
    # Load the .npz file
    data = np.load(file_path)
    predicted_softmax_labels= data['predictions']
    return predicted_softmax_labels

In [11]:
predicted_softmax_file_path = f"output_data/vit_tiny_softmax_predictions_CIFAR10.npz"
predicted_softmax_labels = load_softmax_predictions(predicted_softmax_file_path)

# Processing CLS Embeddings with Distance Metrics

This script processes class embeddings using a specified distance metric (`cosine` or `euclidean`) and evaluates the performance of the predictions.

In [24]:
k_list = [9] #[3,5,7,9,11,13,15,17,19]
use_softmax_predictions = False

for k in k_list:
    print(f'Processing k={k}')
    file_path = f"output_data/cls/cls_neighbors_euclidean_{k}.npz"  # Specify the file path
    test_embeddings, test_labels, all_memory_embeddings, all_memory_labels = load_data_cls(file_path)
    num_classes = len(np.unique(test_labels, return_counts=False))

    print(f"Test CLS: {test_embeddings.shape}, Type: {type(test_embeddings)}")
    print(f"Test Labels: {test_labels.shape}, Type: {type(test_labels)}")
    print(f"Neighbor CLS: {all_memory_embeddings.shape}, Type: {type(all_memory_embeddings)}")
    print(f"Neighbor Labels: {all_memory_labels.shape}, Type: {type(all_memory_labels)}")
    print(f"Distince Number of Classes: {num_classes}")

    # For CLS
    y_pred = []

    distance_metric='cosine'
    #distance_metric='euclidean'

    for selected_test_image_embedding, memory_embeddings, memory_labels, predicted_softmax_label in zip(test_embeddings, all_memory_embeddings, all_memory_labels, predicted_softmax_labels):
        #print(memory_labels)
        if use_softmax_predictions:
            distance_sums = compute_distance_sums(selected_test_image_embedding, memory_embeddings, memory_labels, predicted_softmax_label= predicted_softmax_label, static_distance = 1, distance_metric=distance_metric, num_classes=num_classes)
        else:
            distance_sums = compute_distance_sums(selected_test_image_embedding, memory_embeddings, memory_labels, predicted_softmax_label= None, static_distance = 1, distance_metric=distance_metric, num_classes=num_classes)
        predicted_class = get_predicted_class(distance_sums)
        y_pred.append(predicted_class)
        #break

    # Compute accuracy
    print(classification_report(test_labels, y_pred))

Processing k=9
Data loaded successfully.
Test CLS: (10000, 192), Type: <class 'numpy.ndarray'>
Test Labels: (10000,), Type: <class 'numpy.ndarray'>
Neighbor CLS: (10000, 9, 192), Type: <class 'numpy.ndarray'>
Neighbor Labels: (10000, 9), Type: <class 'numpy.ndarray'>
Distince Number of Classes: 10
              precision    recall  f1-score   support

           0       0.78      0.73      0.76      1000
           1       0.76      0.72      0.74      1000
           2       0.84      0.51      0.63      1000
           3       0.60      0.54      0.57      1000
           4       0.70      0.69      0.69      1000
           5       0.63      0.63      0.63      1000
           6       0.67      0.87      0.76      1000
           7       0.74      0.76      0.75      1000
           8       0.77      0.80      0.78      1000
           9       0.66      0.81      0.73      1000

    accuracy                           0.71     10000
   macro avg       0.71      0.71      0.70     100

# Patch

# `process_lists` Function

The `process_lists` function processes a list of lists by either summing or averaging the elements along the columns. It supports both Python lists and NumPy arrays as input.

In [15]:
def process_lists(lists, mode="sum"):
    if not isinstance(lists, list):
        raise ValueError("Input must be a list of lists.")
    
    lists = [lst.tolist() if isinstance(lst, np.ndarray) else lst for lst in lists]
    
    if not all(isinstance(lst, list) for lst in lists):
        raise ValueError("Each element in the input must be a list.")
    
    if mode not in {"sum", "average"}:
        raise ValueError("Mode must be 'sum' or 'average'.")

    # Convert to NumPy array for vectorized operations
    array = np.array(lists)
    
    if mode == "sum":
        processed = np.sum(array, axis=0)
    else:  # mode == "average"
        processed = np.mean(array, axis=0)
    
    return processed.tolist()  # Return as a list

# `load_data_patch` Function

The `load_data_patch` function loads data from a `.npz` file, which contains test and neighbor class embeddings along with their corresponding labels for patch related info.

In [16]:
def load_data_patch(file_path: str):
    # Load the .npz file
    data = np.load(file_path)
    
    # Extract the arrays
    test_patch = data['test_patch']
    test_labels = data['test_labels']
    neighbor_patch = data['neighbor_path']
    neighbor_labels = data['neighbor_labels']
    print("Data loaded successfully.")
    
    return test_patch, test_labels, neighbor_patch, neighbor_labels

# Patch-Based Prediction and Evaluation

This script processes patch embeddings from test images and memory embeddings to predict classes. It supports two calculation modes (`sum` or `average`) and two distance metrics (`cosine` or `euclidean`). The predictions are evaluated using a classification report.

In [21]:
# For Patch
y_pred = []

calculation_mode = 'average' # 'average'
distance_metric = 'cosine' # 'euclidean'
use_softmax_predictions = True

for selected_test_image_embedding, specific_memory_embeddings, specific_memory_labels, predicted_softmax_label in zip(test_embeddings, all_memory_embeddings, all_memory_labels, predicted_softmax_labels):
    distance_sums_list = []

    for patch_embedding, memory_embeddings, memory_labels in zip(selected_test_image_embedding, specific_memory_embeddings, specific_memory_labels):
        #print(memory_embeddings.shape)
        if use_softmax_predictions:
            distance_sums_list.append(compute_distance_sums(patch_embedding, memory_embeddings, memory_labels, predicted_softmax_label= predicted_softmax_label, static_distance = 1, distance_metric=distance_metric))
        else:
            distance_sums_list.append(compute_distance_sums(patch_embedding, memory_embeddings, memory_labels, predicted_softmax_label= None, static_distance = 1, distance_metric=distance_metric))
    distance_sums = process_lists(distance_sums_list, mode=calculation_mode)
    predicted_class = get_predicted_class(distance_sums)
    y_pred.append(predicted_class)
    

# Compute accuracy
print(classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.57      0.57      1000
           1       0.61      0.56      0.58      1000
           2       0.54      0.36      0.43      1000
           3       0.51      0.27      0.35      1000
           4       0.57      0.45      0.50      1000
           5       0.51      0.52      0.52      1000
           6       0.55      0.70      0.62      1000
           7       0.58      0.69      0.63      1000
           8       0.64      0.70      0.67      1000
           9       0.54      0.83      0.65      1000

    accuracy                           0.57     10000
   macro avg       0.56      0.57      0.55     10000
weighted avg       0.56      0.57      0.55     10000

